In [1]:
import pickle
import pandas as pd
import gensim
import anvil.server
import hm
anvil.server.connect("4JBRYQ2MUR7HBKUJXDUBIWZK-2VXDJRROQ3VSPBLR")

my_root=pd.read_csv('C:/Users/Kois/lemmatizedCorpus.csv') #Lemmatized tokens
my_root=my_root.drop(['Unnamed: 0.1'], axis = 1)
my_root=my_root.drop(['Unnamed: 0'], axis = 1)
my_root=my_root.to_numpy().tolist()

def clean_nan(my_root):
    new_list=[]
    temp_list=[]
    for x in my_root:
        for item in x:
            if str(item) != 'nan':
                temp_list.append(item)
        new_list.append(temp_list)
        temp_list=[]
    return new_list

stop_words=['እኔ', 'የእኔ', 'እኔራሤ', 'እኛ', 'የእኛ', 'እኛራሣችን', 'አንቺ', 'ነህ', 'አላችሁ', 'እርሥዎ', 'ትፈልጋለህ', 'ያንተ', 'ራሥህን', 'እራሣችሁ', 'እሡ', 'የእሡ',
 'ራሡ', 'እሷ', 'እሷናት', 'የእሷ', 'እራሷ', 'ነው', 'እነሡ', 'እነሡን', 'የእነሡ', 'ራሣቸው', 'ምንድን', 'የትኛው', 'ማን', 'ይህ', 'የሚልነው', 'ያ', 'እነዚህ', 'እነዚያ', 'ነኝ', 'ናቸው', 'ነበር', 'ነበሩ', 'ሁን', 'ቆይቷል', 'መሆን',
 'አላቸው', 'አለው', 'ነበረው', 'ያለው', 'መሥራት', 'ያደርጋል', 'አደረገ', 'ማድረግ', 'ሀ', 'አንድ', 'የ', 'እና', 'ከሆነ', 'ወይም', 'ምክንያቱም',
 'እንደ', 'እሥከ', 'እያለ', 'በ', 'ለ', 'ጋር', 'ሥለ', 'ላይ', 'መካከል', 'ወደ', 'በኩል', 'ወቅት', 'ከዚህበፊት', 'በኋላ', 'ከላይ', 'ከታች', 'ከ', 'ወደላይ', 'ታች', 'ውሥጥ', 'ውጭ', 'በላይ',
 'እንደገና', 'ተጨማሪ', 'ከዚያ', 'አንድጊዜ', 'እዚህ', 'እዚያ', 'መቼ', 'የት', 'ለምን', 'እንዴት', 'ሁሉም', 'ማንኛውም', 'ሁለቱም', 'እያንዳንዳቸው', 'ጥቂቶች', 'በጣም', 'ሌላ', 'አንዳንድ',
 'እንደዚህ', 'ብቻ', 'የራሡ', 'ተመሣሣይ', 'ሥለዚህ', 'ይልቅ', 'እንዲሁ', 'ት', 'ይችላል', 'ይገባል', 'ይገባኛል', 'አሁን', 'መ', 'ም', 'ኦ', 'ዳግም',
 'መሆን', 'ሁለ', 'ሁለም', 'ህዝብ', 'ሀሙሥ', 'ለመሆኑ', 'ለምንድን', 'ሌሎች', 'መጽሀፍ', 'ማክሠኞ', 'ምን', 'ሠኞ', 'ሠው', 'ሢሆን', 'ሥንት', 'ረቡእ',
 'ቅዳሜ', 'በዚህ', 'ብላ', 'ነገር', 'አለ', 'አርብ', 'አንተ', 'አንዳንድ', 'ኢትዮጵያ', 'እሁድ', 'እናንተ', 'እንኳን', 'እግር', 'ከመሆን', 'ወይንም', 'ዋና', 'ዘንድ', 'የሚከተለው', 'ያኔ', 'ይኼው', 'ገጽ', 'እነርሡ',
 'ን', 'ና', 'ዎች', 'ይጠበቃል', 'ብለዋል', 'ሆ', 'ሁሉ', 'አንቀጽ', 'እንደሆነ', 'በማይበልጥ', 'መሠረት', 'ሁኔታ', 'ይሆናል', 'ሆኖ', 'ከአንድ', 'በማናቸውም',
 'ወር', 'ከአምሥት', 'በሆነ', 'ከዚህ', 'የሆነ', 'ሀያ', 'ሆነ', 'በኊላ', 'በአንድ', 'የሆኑ', 'ከአሥራ', 'የሆነውን', 'መሆኑ', 'ሌላውን', 'ከሠባት', 'ለሌላ',
 'አለበት', 'ሢል', 'ይሆናሉ', 'በሙሉ', 'አሥራ', 'ቢሆንም', 'አንዱ', 'የሌላውን', 'ከሁለት', 'የሆኑትን', 'በሆኑ', 'ጀምሮ', 'በመሆን', 'ባለ', 'ይህንን', 'እንዲቆይ', 'ሌላው', 'የሚሆነው', 'በአንዱ', 'ሢባል', 'ሣለ', 'የሆነው', 'መሆናቸው', 'በዋና', 'በማቀድ', 'ጊዜና', 'ለዚህ', 'ሦሥተኛ', 'የነገሩ', 'ሥድሥት', 'በሆነው', 'ይሁን', 'ከዚሁ', 'በእነዚህ', 'ከማናቸውም', 'ከነበረው',
 'በአንዳንድ', 'በእያንዳንዱ', 'ጊዜም', 'አሥከ', 'የሌሎች', 'የሚሆኑት', 'ከሆነው', 'የነበረውን', 'ያሉ', 'ከሌሎች', 'አንዲት', 'ለሌሎች', 'ለሆነው', 'ሠኣት', 'ብሎ', 'ከሠላሣ',
 'የሚሆኑ', 'ላይም', 'የሆናል', 'ከነዚህ', 'ያህል', 'ከሆነና', 'ለሆኑት', 'እነዚሁ', 'እንደሆኑ', 'ሥለማናቸውም', 'ሥለዚሁ', 'ከአንዳንድ',
 'በእነዚሁ', 'በአምሥት', 'የሆኑበታል', 'ለነዚህ', 'ለማንኛውም', 'አንደኛ', 'ይኸኛው', 'ከርሡ', 'መሆኑን', 'ለዚያው', 'ለዚሁ', 'ለእነርሡም', 'እዚሁ', 'ሀ', 'ረ', 'ሸ', 'አምሥት',
 'ከሦሥት', 'በተለይም', 'በሌላ', 'ሺህ', 'ማናቸውንም', 'ከአሥር', 'የማይበልጥ', 'እንዲሁም', 'ይህን', 'የዚህ', 'ማናቸውም', 'ከሥድሥት', 'መቶ', 'ያለ', 'አንድን',
 'ያላቸውን', 'ሊሆን', 'ሦሥት', 'ካልሆነ', 'ቢያንሥ', 'ቢሆን', 'እነዚህን', 'አንዱን', 'ሁለት', 'ወይዘሮ', 'ተብሎ', 'ሣይሆን', 'እንደሆነና', 'ከብር', 'ሆኖም', 'የነበሩ',
 'የሌላ', 'ያላቸው', 'ይህንኑ', 'ሆነው', 'በሥተቀር', 'ሥም', 'እንደገና', 'የማያንሥ', 'እጅግ', 'እንዲሆን', 'እንኳ', 'ከሀያ', 'ከሀምሣ', 'ይኸው', 'ለአንድ', 'የሚችለውን',
 'በሚገባ', 'ይህም', 'እንዲሆኑ', 'ከሌላ', 'ለሆነ', 'በሌሎች', 'አንደሆነ', 'እንዲህ', 'በነዚሁ', 'በእንደዚህ', 'ሥምንት', 'ሢሆንና', 'ምንጊዜም', 'ለማናቸውም', 'የአንድ', 'እነዚህኑ', 'ሢሆኑ', 'በሁለቱም',
 'እንደነዚህ', 'የሆኑት', 'የማናቸውም', 'ይህንንም', 'የአንድን', 'በሙሉም', 'በነዚህ', 'የዚሁ', 'ለእያንዳንዱ', 'ሥለሆነ', 'መሆናቸውን', 'ማንኛውንም', 'ሁለቱ', 'እንጂ',
 'ከሥምንት', 'ሁለቱንም', 'በሁለት', 'በእሥር', 'በሚል', 'ቁጥር', 'ባሉ', 'ከመቶ', 'እነዚህም', 'ሢኖር', 'ሠላሣ', 'ለሆኑ', 'ሠባት', 'እነደሆነ', 'ይህችው', 'ከእነዚህ',
 'ከእነዚሁ', 'የአንቀጹ', 'ወይ', 'የሆነችን']

def stopWordRemoval(tokens):
    token_new=[]
    for x in tokens:
        if x not in stop_words:
            token_new.append(x)
    
    return token_new


def build_stopWord_free_corpus(new_list):
    processed_docs=[]
    for item in new_list:
        processed_docs.append(stopWordRemoval(item))
    
    return processed_docs


def remove_verbs(processed_docs):
    listOfVerbs="ተሰለፈ,ድርግ,ሰጠ,ገባ,አወቀ,መሰለ,መጣ,ጠማ,ነደደ,ወጣ,ለየ,ጣለ,ነዳ,ኖረ,ተከለ,ፈለገ,ወሰነ,ቻለ,ተገኛኘ,መታ,ሰረረ,ባከነ,አነሰ,ደገመ,ያዘ,አነሳ,አለቀ,አስቀመጠ,ለበሰ,ላከ,አወገዘ,አካሄደ,ገደለ,አለፈ,ቀረረ,ለቀቀ,ለመጠ,ጀመረ,ተናገረ,ገመተ,ዶለ,ፈጠረ,ሰቀቀ,ፈጸመ,ቀረ,አየ,ቈየ,ዘገበ,በላ,ተባለ,ፈላ,ቀነተ,ፈታ,ሞተ,ገለጸ,ገለጠ,ደበነ,ሰደደ,ጐለተ,ተቈጣ,ማረ,አፈረ,ዘረዘረ,ጠበቀ,ተወ,መረጠ,ተቀበለ,ተመቸ,ደረደረ,ተመለከተ,ጨፈነ,ዞረ,ዠለጠ,ነፈሰ,ተሸነፈ,አማ,ረጋ,ተቃወመ,ቀየረ,አለመ,ገገረ,ቀለለ,ተዘጋጀ,ዋለ,ቆመ,ተላለፈ,ቀነበረ,ጨረሰ,ቋጠረ,ተሰራጨ,አራ,ሾመ,ስምር,ተራመደ,ቀፈፈ,ተቋቋመ,ረገበ,ሰከረ,ለቀለቀ,ታገለ,አጐነበሰ,አሰረ,ረገጠ,ተጠረጠረ,ነካ,ተነሳ,አበለ,ጠቀመ,አረገ,ጠፋ,ወደቀ,ነዛ,ደረሰመ,ገረረ,ቀጠፈ,ረሳ,ፈጀ,ሰለቸ,ለወጠ,ደከመ,ሄደ,ሰደበ,ቀላ,ቀረጸ,ፎከረ,ወቀረ,ተማረ,ወለደ,ሳበ,ሰራ,ቀደሰ,አደረ,ለመደ,ተረጐመ,ሰፈነ,ፈነጠቀ,ቀለደ,ገበየ,ወቃ,አመሰ,አጣ,ዘፈነ,ተደረገ,ወሰደ,ነጐደ,ቀቀለ,አናገረ,በሳ,መጠነ,ተካሄደ,ተንጠባጠበ,ረዘመ,መከረ,መሰከረ,ከለከለ,ፈራ,ወደደ,ደገፈ,ተጫወተ,ለመለመ,ተሰቃየ,ዘና,ተሸበረ,ገነባ,ጨበጨበ,ቈረቈረ,ቈረጠ,ጨቈነ,ቀረበ,ተሳበ,ሸጠ,ገዛ,ላላ,ባሰ,ተቀመጠ,ቀጠለ,ደፈነ,ተመለከ,ቀደመ,ፈሳ,ተመኘ,ነቃ,መጠጠ,ሮጠ,ተመካ,ቀደደ,ተከተለ,ተወራ,ናደ,ቀጣ,ገታ,ቀሰቀሰ,ተሻገረ,ጠየቀ,ፋቀ,ጣረ,ተደሰተ,ለመነ,ወተወተ,ቀማ,በጠበጠ,ዋጠ,ተከበረ,ቸገረ,ተቀየመ,ተጠቃ,አረመ,ዘረፈ,ጠራ,ሳቀ,አስተማረ,ጠረገ,አመጠ,ተያያዘ,ጻፈ,ገረመ,አከከ,ደረቀ,ተንበረከከ,ተለጐመ,ከሳ,ተጨነቀ,ዋሸ,ተታለለ,ሰፈረ,ፈተነ,በረከተ,ተከላከለ,ተሳተፈ,ገፋ,ተመሰገነ,ሰለጠነ,ተንቀሳቀሰ,ተወዳጀ,ጨፈረ,ተመጻደቀ,ተባበረ,ጠነከረ,ተቀዳጀ,ተንፈረጠጠ,ጠቀሰ,ቈጠረ,ዘረጋ,ጠቀለለ,ባረከ,ሳተ,ሸረሞጠ,ላጠ,ዳጠ,ነጠቀ,ተከናወነ,ላገ,በደለ,ጨበጠ,ዘጋ,ጨቀጨቀ,ላፈ,ነሳ,ወረረ,ዳነ,ዘከዘከ,አቀደ,ተገለገለ,ተመነ,አመመ,ገጠመ,ተመረኰዘ,ተዳረሰ,ታጠቀ,አፈነ,ተንከባከበ,ኰነነ,ተቃጠለ,አረደ,ፈረሰ,ተደራጀ,ጠለዘ,ቀረጠፈ,ተዘናጋ,ሻ,ከሰሰ,ረገመ,ተገደደ,ተጨናነቀ,ጠረቀመ,ቀነሰ,ረካ,ዳበረ,ጐለበተ,አለበ,ወረወረ,ጨፈለቀ,ተኛ,ደበቀ,ቈረጠመ,መዘገበ,ላመ,ወከለ,ነቀለ,ኰሳ,ሸለለ,አሸ,ጠዳ,ተተራመሰ,ዘመተ,ጸደቀ,ከተመ,ሰካ,ታወረ,ተሳካ,ነጣ,አገዘ,ገደበ,ተሳሳተ,ለፋ,ተበላሸ,ሸመቀ,ናፈቀ,ሰበሰበ,ጠለፈ,ተከፋ,ተሻሻለ,ከሰመ,ፈቀደ,ተጋ,ጐደለ,ቈጨ,ነባ,አስቻለ,በረረ,ከፈተ,አለለ,ጠመመ,ወጠረ,ገበረ,ታታ,ገለበጠ,ሸመገለ,ደነበረ,ተደናቀፈ,ሰላ,መረረ,ጠቈመ,ጋበዘ,ቀበረ,ጫነ,ናቀ,ሰወረ,ቀሰረ,ተቸ,መረመረ,መዘነ,ወደመ,ተወዳደረ,ተከባበረ,ተዳደረ,ተሰኘ,ጋረጠ,ፈሰሰ,ገነዘ,አደለ,አሸገ,ደፋ,ጸለየ,ደወለ,ጠጣ,ተጠጋ,ዳመጠ,መሰገ,ቀላቀለ,ማለ,ዘመረ,ሰየመ,ጣሰ,ለቀመ,አከፋፈለ,ጸነነ,በጠሰ,ጣፈ,ታወሰ,አለቀሰ,አበደ,ዘራ,አጨደ,ተጨማለቀ,ጮኸ,ከሰከሰ,መሸ,ታበየ,ጋረ,ማረረ,አጽናና,ተግባባ,መነጨ,አጠበ,ረጨ,ለፈለፈ,አደሰ,ሞገተ,አቻለ,ወረሰ,ሞከረ,ተሳነ,ወፈረ,ተከዘ,ነከሰ,አረረ,ተዛዘበ,ሸተተ,ፈነቀለ,ወቀሰ,ሳሳ,ዘነጋ,ገራ,ተበጀ,ካሰ,ደነዘዘ,ረበረበ,ጸዳ,ተከታተለ,ተገነዘበ,አዘነበለ,ተወሀደ,ራቀ,ዘነነ,ገጨ,ከተበ,ሸረሸረ,አለቃቀሰ,ተነገረ,ተላላከ,ቀዳ,ጠባ,ፈነዳ,ተወናበደ,ተስተካከለ,ጠበበ,ቈሸሸ,ተጠነቀቀ,ደነገጠ,ተጓዘ,አቀፈ,መገበ,ፈካ,ፈወሰ,ሻረ,ዛለ,ተከራየ,አስተዋለ,ጠበሰ,ተኰሰ,ተጠና,ተነፈሰ,በተነ,ቈፈረ,ጠረበ,ተደመመ,ዘቀጠ,ተዋከበ,ቃመ,ናወዘ,ተልከሰከሰ,አርጋጋ,ተሽከረከረ,ተለተመ,ተጣደፈ,ዋተተ,ተገለለ,ዶለተ,ነፈገ,ተጠናቀቀ,በሰለ,ደመደመ,አከተ,ሰጋ,ማረከ,በረቀ,ቈለለ,ማቀቀ,ተወገደ,ተኰላሸ,ዋጀ,ተረተ,ተስማማ,ረባ,ፈጠጠ,ፋረ,ሰነጠቀ,ደበደበ,ደፈረሰ,ተጠናወተ,ቀሰመ,ኰላ,አጠፈ,ጣመ,አፈሰ,ነፋ,ረፈደ,ተዋረደ,ሰነዘረ,ተቋረጠ,አወጋ,ተንጠለጠለ,አረሰ,ከለበ,ተከራከረ,ፈተሸ,አተመ,ቈሰለ,ተደለደለ,ገመገመ,ተነጻጸረ,ተዋወቀ,ተሸከመ,አረቀ,አገደ,ተቻቻለ,ሰመጠ,ወጋ,ተደመጠ,ፈለፈለ,ጨከነ,ከተተ,ዘባረቀ,መረተ,አመለጠ,ነከተ,ተመቻቸ,ሸፈነ,ጠመዘዘ,ሰበረ,ሰረቀ,ወለቀ,አነበበ,ገረፈ,መረቀ,አጠረ,ተምታታ,ሰበከ,ዳረ,ሰዋ,ለኰሰ,ተካረረ,አደነ,ራሰ,ቀረፈ,ሰለመ,ጠነባ,ሰገደ,ጐበኘ,ከረፋ,አኘከ,ወነጀለ,ተጣመረ,ጓጓ,ዘነጠ,ደበለቀ,ፈዘዘ,አጀበ,ከተፈ,መደበ,ቀጠረ,ተለማመጠ,ተበደረ,ጠፈፈ,ታገሰ,አነቀ,ሰነበተ,ደረተ,ሰቀለ,ደፈረ,ዘለፈ,ጨፈጨፈ,ረቀቀ,ተነጋገረ,ተከፈፈ,ተሰናበተ,ላሰ,ሰለበ,ዳሰሰ,ራበ,ሸኘ,ቀባጠረ,ተርበተበተ,አመሸ,ተሳለቀ,ቸከ,ገዘገዘ,ዳከረ,ገዘፈ,ጐተተ,ጋጠ,ጾመ,ወቀጠ,ተሞገሰ,ጠገበ,ከጀለ,ተሸቀበ,ረሸነ,ተረከበ,ዘገነነ,ከረከረ,ተሞሸረ,ፈረመ,ሸመተ,ከበተ,አስመለሰ,ፈለጠ,ረበሸ,ደጐመ,ተነጣጠረ,ሰረሰረ,ገላገለ,ተዛመደ,ተጃጃለ,በሰበሰ,ዋለለ,ተበሳጨ,ረጠበ,ተንጨረጨረ,ተረከ,ኰለኰለ,ካነ,አረጠ,ካደ,ወዘወዘ,ረበበ,ተቈጣጠረ,ተጸጸተ,አላመጠ,አካፈለ,ሸመጠጠ,አኰረፈ,ለጠፈ,አወከ,ገነጠለ,ጐነጐነ,ቀለመ,ተለመ,ቈጠበ,ዛገ,ነደለ,ገመደ,ሰገሰገ,ጋረደ,ተጠተተ,መረዘ,ደረገመ,ሸና,ተጸነሰ,ጨነገፈ,አስተናገደ,ሸቀጠ,ነቀነቀ,ተቃረነ,ተመለጠ,ተፋ,አሾፈ,ሸሸ,ጠረዘ,ተበላለጠ,ቀባ,ደለበ,ዋኘ,ፈረጠ,ከለሰ,በረደ,ተሸማቀቀ,ቀጠቀጠ,ሰነገ,ተግደረደረ,ቀየሰ,ጫረ,አነጠሰ,ተዘዋወረ,ተካፈለ,ተጥለቀለቀ,ተጭበረበረ,ሰከነ,ተዘወረ,ተንቀጠቀጠ,ጠመጠመ,ሰረዘ,አስተጋባ,ከሰረ,ጠለቀ,ተንከረፈፈ,ዘመመ,ሞቀ,ዳሰ,ፈነጠረ,ተለቀ,ቀበጠ,ተገናዘበ,ከሸፈ,ተነተነ,ዳረገ,ቃኘ,ገፈፈ,ጃጀ,ሰቀጠጠ,ተፍጨረጨረ,ነደፈ,ጋለ,ተንሳሳ,ደገሰ,ጀገነ,ቀለጠ,ዛተ,መለለ,ተባባለ,ተስፈነጠረ,ጠመደ,አነከሰ,ጠመቀ,ተገመሰ,ለጠቀ,ማገ,ቋጨ,ተዋሰ,ገሰገሰ,ወረፈ,ተቀኘ,ተነኰተ,ቀላወጠ,ሰየፈ,ጨለፈ,ቸረ,ተረተረ,በከለ,ተወላከፈ,አመላከተ,ከለለ,አጐረ,መነመነ,አበሰ,ደገነ,ሰነፈ,ተጠየፈ,ተነገተ,ወገረ,ቻቻለ,ቀጨጨ,ከረከመ,ራደ,ተኰፈሰ,ተነጠለ,ታጨ,ተንጫጫ,ወሰለተ,ወሸቀ,ኰሰመነ,ተነጫነጨ,ዘረገፈ,ተንሳፈፈ,ተፋጨ,ሾለ,ተንፈራፈረ,ጐረበጠ,ጠቈረ,ቀመሰ,ነተበ,ደላ,ፈረጠጠ,ደነቈረ,መነጠረ,ደመሰሰ,ተዛወረ,ሳገ,ተባ,ሻጠ,ሳመ,ኰረጀ,ረከሰ,ቀዘነ,ከሸነ,በዘበዘ,ተፋለመ,ተደለቀ,ተፈናጠጠ,ከካ,ወዛ,ፈነጨ,አጠነ,ዳበሰ,አመራ,ረገፈ,ገለበ,ፈየደ,ተራቈተ,ጐሰቈለ,ሸቀለ,ሰበቀ,ተቅነዘነዘ,ሰለቀ,ለዘበ,ደሰመ,አሴረ,ተጋዘ,ለሰለሰ,ቀሰፈ,ተንከራተተ,ማገደ,ፈነከተ,ዘላበደ,ዘበዘበ,ተንደላቀቀ,ተደረሰ,ሸረፈ,ተቅለሰለሰ,ተላከከ,ተሻማ,ደፈቀ,ነጠረ,ዘረረ,ደለለ,ተቀጣጠለ,ተውተረተረ,ጐረፈ,ተደረበ,ቸለሰ,ተንደረደረ,ተናጠ,ተከማቸ,ፈተፈተ,ተንሸራተተ,ቀጠነ,ተትረፈረፈ,ኳለ,ጨለመ,ነፈረ,ከሰለ,ተፋጀ,ተዳበለ,ወጠነ,አጋመሰ,ለበለበ,ሸመቀቀ,ተካተተ,ተዋበ,አከመ,ኰረፈ,በገነ,ደበዘዘ,ጠበጠበ,ተቃቃመ,ጋመ,መነዘረ,ዘየደ,ወረዛ,ቀነጠሰ,ነቀሰ,ተገዘተ,ተረባረበ,ጐበዘ,ለጠጠ,ደደበ,ተንጣለለ,ተንቀለቀለ,ሰጠመ,ታከከ,ተቈራ,ጠገነ,ቦረቀ,ከተከተ,ተሽኰረመመ,አመለከተ,ተቀለበ,ለሰነ,ተቈራኘ,ሞገደ,ፈነደቀ,መሰነ,ተዝረከረከ,ሸመደደ,ተንበለበለ,ዘበተ,ቦዘ,ገነፈለ,ነቀፈ,ተጸየፈ,ተደላደለ,ደሰሰ,ነዘረ,ሳለ,ቀዘቀዘ,ነዘነዘ,ደደረ,ተበጠረ,ባተ,ተመሳቀለ,አጠራጠረ,ገነገነ,ነከረ,ተሳደደ,አለጠ,ደቀደቀ,መተረ"
    listOfVerbs=listOfVerbs.split(",")
    no_verb=[]
    verb_set=set(listOfVerbs)

    for item in processed_docs:
        item_set=set(item)
        no_verb.append(list(item_set.difference(verb_set)))
    
    return no_verb

def remove_new_verbs(list_of_terms):
    no_verbs=[]
    for item in list_of_terms:
        try:
            y=hm.anal('amh',item,um=True)
            if y[0]['POS']!='v':
                no_verbs.append(item)
        except:
            no_verbs.append(item)
    return no_verbs

new_list=clean_nan(my_root)
processed_docs=build_stopWord_free_corpus(new_list)
processed_docs=remove_verbs(processed_docs)

dictionary = gensim.corpora.Dictionary(processed_docs)
dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=100000)
corpus = [dictionary.doc2bow(doc) for doc in processed_docs]




@@@@ This is HornMorpho, version 4.0.5 @@@@

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment" as SERVER


In [ ]:
@anvil.server.callable
def extract_target_lda(text):
    list_of_terms=text.split(" ")
    list_of_terms=stopWordRemoval(list_of_terms)
    list_of_terms=remove_new_verbs(list_of_terms)
    
    ldamodel = pickle.load(open('C:/Users/Kois/LDA_Code/LDA_Code/pickle lda/LDA_k20_a10_b0_c0.4707876629910356.pkl', 'rb'))
    predictions=ldamodel.get_document_topics(dictionary.doc2bow (list_of_terms))
    top_topic=sorted(predictions, key=lambda t:t[1], reverse=True)
    t=top_topic[0:1]
    top_topic_id=t[0][0]
    if t[0][1]>=0.05:
        return(ldamodel.print_topics()[top_topic_id])
    else:
        return 0

In [ ]:
extract_target_lda("መሪ ልክ ቦታ ትምህርት ተሻለ")

text="ሞጣ በደረሰው አደጋ አዝኛለው"
list_of_terms=text.split(" ")
list_of_terms=stopWordRemoval(list_of_terms)
list_of_terms=remove_new_verbs(list_of_terms)

ldamodel = pickle.load(open('LDA_k20_a10_b0_c0.4707876629910356.pkl', 'rb'))
predictions=ldamodel.get_document_topics(dictionary.doc2bow (list_of_terms))
top_topic=sorted(predictions, key=lambda t:t[1], reverse=True)
t=top_topic[0:1]
for idx, topic in ldamodel.print_topics(-1):
    if idx ==t[0][0]:
        print(topic)

In [2]:

ldamodel = pickle.load(open('LDA_k20_a4_b1_c0.47855140175765654.pkl', 'rb'))
            
for idx, topic in ldamodel.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.109*"ሰላም" + 0.082*"ብልጽግና" + 0.054*"ፍቅር" + 0.054*"ጥያቄ" + 0.052*"መለሰ" + 0.040*"አብይ" + 0.029*"ምክንያት" + 0.023*"ጀግና" + 0.019*"ፎቶ" + 0.019*"ህወሀት"
Topic: 1 
Words: 0.089*"አመት" + 0.078*"ቀን" + 0.058*"ገና" + 0.043*"ከበረ" + 0.037*"ሀይል" + 0.036*"በቃ" + 0.034*"አሉ" + 0.025*"ልደት" + 0.025*"ለውጥ" + 0.024*"ደም"
Topic: 2 
Words: 0.113*"መንግስት" + 0.088*"ዛሬ" + 0.068*"ፓርቲ" + 0.064*"መራ" + 0.043*"አገር" + 0.031*"ዝም" + 0.023*"ስርአት" + 0.023*"ድምጽ" + 0.021*"መብት" + 0.020*"ስብሰባ"
Topic: 3 
Words: 0.114*"ደረሰ" + 0.058*"አዲስ" + 0.050*"ዘመን" + 0.043*"አባት" + 0.041*"አለም" + 0.041*"ሰአት" + 0.036*"መሪ" + 0.032*"ነጻ" + 0.027*"መረጃ" + 0.025*"አሜሪካ"
Topic: 4 
Words: 0.110*"አማራ" + 0.109*"ተገኘ" + 0.086*"ክልል" + 0.051*"ሙስሊም" + 0.040*"ከፈለ" + 0.039*"ተቻለ" + 0.032*"ድርጅት" + 0.029*"ትንሽ" + 0.024*"ሞጣ" + 0.019*"ሙሉ"
Topic: 5 
Words: 0.054*"ሰው" + 0.049*"ህግ" + 0.045*"ቦታ" + 0.042*"እውነት" + 0.033*"ትልቅ" + 0.029*"ክብር" + 0.029*"ህጻን" + 0.027*"ወያኔ" + 0.024*"ንብረት" + 0.022*"ሞላ"
Topic: 6 
Words: 0.080*"ለም" + 0.068*"ልብ" + 0.046*"ግልጽ" + 0.040*"በለጠ" + 0.

topic_id=[]
for i in t:
    topic_id.append(i[0])
for idx, topic in ldamodel.print_topics(-1):
    if idx in topic_id:
        print(topic)

ldamodel = pickle.load(open('LDA_k20_a10_b0_c0.4707876629910356.pkl', 'rb'))
t=ldamodel.get_document_topics(dictionary.doc2bow (['አይነት','አገር','ጠላት']))
t=sorted(t, key=lambda t:t[1], reverse=True)
t=t[0:1]

ldamodel.get_document_topics(dictionary.doc2bow (['አይነት','አገር','ወደደ','ጠላት','ወደደ','ድርግ','ጠላት','ፈለገ','እርግፍ','ድርግ','ስብእና','ፈጠረ','ገነባ','ማንነት','ደረደረ','ጠነከረ','ሰራ','ተጋላጭነት','ቀነሰ','ተቻለ','አየ','ጉዳይ','አዋጭነቱ','ነዉ']))

In [ ]:
# dictionary.doc2bow (['አብቹ','ለለውጥ','ምሳሌ','መሪ','ደግሞ','የሁላችንም','ሜዳ','ገብቶ','መጫወት','የሚፈልግ','ሂደት','ሁላችንም','የመሪያችንን','መልካም','ተግባራት','ተከትለን','ለሃገራችን','ብልጽግና','እንስራ'])
# dictionary.doc2bow (['“በመወለዱ','ብዙዎች','ደስ','ይላቸዋል”','ሉቃስ','ቅዱስ','ደግ','አቡነ','ተክለሃይማኖት','ታህሳስ','ተወለዱ','አባታችን','ረድኤታቸው','በረከታቸው','አማላጅነታቸው','አይለየን','ፈጣሪ','ሆይ','ህዝቦቿን','በቸርነትህ','ይጠብቅልን'])

topic_probability=ldamodel.get_document_topics([(80, 1), (146, 1), (164, 1), (180, 1), (249, 1), (252, 1), (357, 1), (468, 1), (601, 1), (874, 1), (1700, 1)])
topic_probability

###

# Target Extraction

In [ ]:
test_set=pd.read_csv('C:/Users/Kois/LDA/36Tokenizedsentences.csv')
test_set=test_set.drop(['Unnamed: 0'], axis = 1)
test_set=test_set.to_numpy().tolist()
new_list=[]
temp_list=[]
for x in test_set:
    for item in x:
        if str(item) != 'nan':
            temp_list.append(item)
    new_list.append(temp_list)
    temp_list=[]
test_set=new_list

def lemmatizer(tokenList):
    lemmatized_list=[]
    for x in tokenList:
        try:
            y=hm.anal('amh',x,um=True)
            b=y[0]['lemma'].split("|")
            lemmatized_list.append(b[0])
        except:
            print('next')
    return lemmatized_list
count=0
test_set_doc=[]
for x in test_set:
    temp=(stopWordRemoval(x))
    test_set_doc.append((temp))
    count=count+1
    #print(count)

In [ ]:
#  ldamodel.get_document_topics(dictionary.doc2bow())
test_sentence_text=pd.read_csv('C:/Users/Kois/Downloads/100 sentences.csv')
topic_prob=[]

for doc in test_set_doc:
    topic_prob.append(ldamodel.get_document_topics(dictionary.doc2bow(doc)))
    
for t,y in zip(topic_prob,test_sentence_text['tweet']):
    print(y,sorted(t,key=lambda t:t[1], reverse=True))

In [ ]:
test_sentence_text=pd.read_csv('C:/Users/Kois/Downloads/100 sentences.csv')
test_sentence_text['tweet']

# setup: get the model's topics in their native ordering...
all_topics = ldamodel.print_topics()
# ...then create a empty list per topic to collect the docs:
docs_per_topic = [[] for _ in all_topics]
count=0
# now, for every doc...
for doc_id, doc_bow in enumerate(corpus):
    # ...get its topics...
    doc_topics = ldamodel.get_document_topics(doc_bow)
    # ...& for each of its topics...
    for topic_id, score in doc_topics:
        # ...add the doc_id & its score to the topic's doc list
        docs_per_topic[topic_id].append((doc_id, score))

print(docs_per_topic[11])